In [1]:
from pymongo import MongoClient, ASCENDING
from neo4j import GraphDatabase
import pandas as pd

RETO 1 – CARGA Y EXPLORACIÓN 
1. Revisa los ficheros adjuntos de la actividad y la documentación 
correspondiente de cada fichero. 
2. Realiza una exploración de los ficheros utilizando MongoDB como motor de 
base de datos. Para ello: 
a. Carga los datos en la BD (utiliza los ficheros CSV o JSON) 
b. Revisa los campos y comprueba la calidad de los datos en cada uno de 
los ficheros. 
c. Encuentra un campo que permita relacionar los ficheros entre sí. 

Apartado A

Insertar los ficheros en cada uno de las colecciones

In [14]:
myclient = MongoClient("mongodb://localhost:27017/")
mydb = myclient["tarea_nosql"]
mycol_economica = mydb["actividad_economica"]
mycol_licencias = mydb["licencias"]
mycol_locales = mydb["locales"]
mycol_terrazas = mydb["terrazas"]

Apartado B

compruebo la calidad de la coleccion actividad economica

In [3]:
x = mycol_economica.find_one({"id_local" : 20})
e = mycol_licencias.find_one({"id_local" : 7})
y = mycol_locales.find_one({"id_local" : 2})
o = mycol_terrazas.find_one({"id_local" : 270403150})

Validar los datos de las colecciones

In [4]:
mydb.command("validate", "actividad_economica")
mydb.command("validate", "licencias")
mydb.command("validate", "locales")
mydb.command("validate", "terrazas")

{'ns': 'tarea_nosql.terrazas',
 'uuid': Binary(b'D\x91h@\x0e\xf4@\xd2\x84F\x0f\xb5\xcfat\x07', 4),
 'nInvalidDocuments': 0,
 'nNonCompliantDocuments': 0,
 'nrecords': 6788,
 'nIndexes': 2,
 'keysPerIndex': {'_id_': 6788, 'id_local_1': 6788},
 'indexDetails': {'_id_': {'valid': True}, 'id_local_1': {'valid': True}},
 'valid': True,
 'repaired': False,
 'warnings': [],
 'errors': [],
 'extraIndexEntries': [],
 'missingIndexEntries': [],
 'corruptRecords': [],
 'ok': 1.0}

Apartado C
El campo que permite relcionar entre colecciones es el id_local

RETO 2 – MODELADO DE DATOS 
Después de explorar todos los ficheros y tener clara su estructura, resuelve los 
siguientes tareas. 
Modelo de datos (versión 1) 
1. Propón un modelo de datos basado en documentos que permita almacenar la 
información de los ficheros y consultarla posteriormente. Este modelo 
propuesto puede seguir un patrón de datos visto en este curso (embebidos, 
referenciados o jerárquicos). 
a. Crea una base de datos en MongoDB y almacena los ficheros siguiendo 
tu modelo de datos. 
2. Para mostrar la eficiencia de tu modelo de datos, construye y ejecuta las 
siguientes consultas. 
a. El total de Locales y Terrazas por Distrito y Barrio. 
b. Tipos de licencias y cantidad de licencias por cada tipo.  
c. Listado de locales y terrazas con licencias “En trámite”. 
d. Consulta por sección, división y epígrafe de la actividad comercial de 
locales y terrazas. 
e. Consultar la actividad económica más frecuente por barrio y distrito. 
f. Actualiza los horarios de apertura y cierra de “ciertos” locales 
siguiendo un criterio que tú elijas (explica bien dicho criterio en el 
informe final). 

Modelo de datos (Versión 1)


Crear el indice

In [ ]:
mycol_locales.create_index([('id_local')])

In [5]:
pipeline = [
  {"$lookup": {
    "from": "actividad_economica",
    "localField": "id_local",
    "foreignField": "id_local",
    "as": "actividad_economica"
  }},
  {"$lookup": {
    "from": "licencias",
    "localField": "id_local",
    "foreignField": "id_local",
    "as": "licencias"
  }},
  {"$lookup": {
    "from": "terrazas",
    "localField": "id_local",
    "foreignField": "id_local",
    "as": "terrazas"
  }},
  {"$out": "locales_embebidos"}
]
mycol_locales.aggregate(pipeline)

a. El total de Locales y Terrazas por Distrito y Barrio.

In [5]:
mycol_locales_embebido = mydb["locales_embebidos"]

In [6]:
resultado = mycol_locales_embebido.aggregate([
    {"$group": {
        "_id": {
            "distrito": "$desc_distrito_local", "barrio": "$desc_barrio_local"
            },
        "total_locales": {
            "$sum": 1
            },
        "total_terrazas": 
        {"$sum": 
         {"$size": "$terrazas"}
        }
    }}
])


for grupo in resultado:
    print(grupo)

b. Tipos de licencias y cantidad de licencias por cada tipo.

In [ ]:
resultado = mycol_locales_embebido.aggregate([{"$unwind": "$licencias"},{"$group":
                                               {"_id": "$licencias.desc_tipo_licencia", "total": {"$sum" : 1}
                                                }}])
for grupo in resultado:
    print(grupo)

['ARGANZUELA          ',
 'BARAJAS             ',
 'CARABANCHEL         ',
 'CENTRO              ',
 'CHAMARTIN           ',
 'CHAMBERI            ',
 'CIUDAD LINEAL       ',
 'FUENCARRAL-EL PARDO ',
 'HORTALEZA           ',
 'LATINA              ',
 'MONCLOA-ARAVACA     ',
 'MORATALAZ           ',
 'PUENTE DE VALLECAS  ',
 'RETIRO              ',
 'SALAMANCA           ',
 'SAN BLAS-CANILLEJAS ',
 'TETUAN              ',
 'USERA               ',
 'VICALVARO           ',
 'VILLA DE VALLECAS   ',
 'VILLAVERDE          ']

c. Listado de locales y terrazas con licencias “En trámite”. 

In [ ]:
resultado = mycol_locales_embebido.find({"licencias": {"$elemMatch": {"desc_tipo_situacion_licencia": "En tramitación"}}})

for grupo in resultado:
    print(grupo)

d. Consulta por sección, división y epígrafe de la actividad comercial de 
locales y terrazas. 

In [ ]:
resultado = mycol_locales_embebido.aggregate([
    {"$unwind": "$actividad_economica"},
    {"$group": {
        "_id": {
            "seccion": "$actividad_economica.desc_seccion",
            "division": "$actividad_economica.desc_division",
            "epigrafe": "$actividad_economica.desc_epigrafe"
        },
        "total_locales": {"$sum": 1},
        "total_terrazas": {"$sum": {"$size": "$terrazas"}}
    }}
])

for grupo in resultado:
    print(grupo)

e. Consultar la actividad económica más frecuente por barrio y distrito. 

In [ ]:
resultado = mycol_locales_embebido.aggregate([
    {"$unwind": "$actividad_economica"},
    {"$group": {
        "_id": {
            "distrito": "$desc_distrito_local",
            "barrio": "$desc_barrio_local",
            "actividad": "$actividad_economica.desc_epigrafe"
        },
        "count": {"$sum": 1}
    }},
    {"$sort": {"count": -1}},
    {"$group": {
        "_id": {
            "distrito": "$_id.distrito",
            "barrio": "$_id.barrio"
        },
        "actividad_mas_frecuente": {"$first": "$_id.actividad"},
        "count": {"$first": "$count"}
    }}
])

for grupo in resultado:
    print(grupo)

f. Actualiza los horarios de apertura y cierra de “ciertos” locales 
siguiendo un criterio que tú elijas (explica bien dicho criterio en el 
informe final). 

In [ ]:
mycol_locales_embebido.update_many(
    {"hora_apertura1": {"$lt": "08:00"}},
    {"$set": {"hora_apertura1": "08:00"}}
)

Modelo de datos (versión 2) 

3. Ahora que ya cuentas con un modelo de datos sobre la actividad económica de 
Locales y Terrazas en la ciudad de Madrid, vamos a necesitar conocer la 
actividad relacionada con los alojamientos de los distintos barrios de esta 
misma ciudad. Para incluir esta información, debes realizar lo siguiente: 
a. Revisa la siguiente URL y localiza información sobre los alojamientos 
de Airbnb http://insideairbnb.com/new-york-city/ de la ciudad de 
Madrid. 
b. Debes encontrar un dataset y cargar dicha información en la base de 
datos donde alojas tu modelo de datos. 
c. Asocia la información de alojamientos con la actividad comercial, 
propón algún tipo de mejora de tu modelo de datos para contemplar la 
nueva información.

A: Dataset Localizado

B: Dataset Listening

b. Debes encontrar un dataset y cargar dicha información en la base de 
datos donde alojas tu modelo de datos.

In [16]:
coleccion_alojamientos = mydb['airbnb']

In [10]:
res = coleccion_alojamientos.find_one()
print(res)

{'_id': ObjectId('65c8fbf5ff639b3d17edf29e'), 'id': 6369, 'name': 'Rental unit in Madrid · ★4.88 · 1 bedroom · 1 bed · 1 private bath', 'host_id': 13660, 'host_name': 'Simon', 'neighbourhood_group': 'Chamartín', 'neighbourhood': 'Hispanoamérica', 'latitude': 40.45724, 'longitude': -3.67688, 'room_type': 'Private room', 'price': 85, 'minimum_nights': 5, 'number_of_reviews': 106, 'last_review': datetime.datetime(2023, 10, 16, 0, 0), 'reviews_per_month': 0.63, 'calculated_host_listings_count': 1, 'availability_365': 22, 'number_of_reviews_ltm': 4}


In [11]:
pipeline = [
    {
        '$lookup': {
            'from': 'locales_embebidos',  # nombre de la colección con la que quieres hacer el JOIN
            'localField': 'neighbourhood',  # campo de la colección de alojamientos
            'foreignField': 'desc_barrio_local',  # campo de la colección de locales
            'as': 'locales_en_mismo_barrio'  # nombre del nuevo campo que contendrá los documentos relacionados
        }
    }
]

resultados = coleccion_alojamientos.aggregate(pipeline)

In [12]:
for resultado in resultados:
    print(resultado)

{'_id': ObjectId('65c8fbf5ff639b3d17edf29e'), 'id': 6369, 'name': 'Rental unit in Madrid · ★4.88 · 1 bedroom · 1 bed · 1 private bath', 'host_id': 13660, 'host_name': 'Simon', 'neighbourhood_group': 'Chamartín', 'neighbourhood': 'Hispanoamérica', 'latitude': 40.45724, 'longitude': -3.67688, 'room_type': 'Private room', 'price': 85, 'minimum_nights': 5, 'number_of_reviews': 106, 'last_review': datetime.datetime(2023, 10, 16, 0, 0), 'reviews_per_month': 0.63, 'calculated_host_listings_count': 1, 'availability_365': 22, 'number_of_reviews_ltm': 4, 'locales_en_mismo_barrio': []}
{'_id': ObjectId('65c8fbf5ff639b3d17edf29f'), 'id': 30320, 'name': 'Rental unit in Madrid · ★4.63 · 1 bedroom · 2 beds · 1 bath', 'host_id': 130907, 'host_name': 'Dana', 'neighbourhood_group': 'Centro', 'neighbourhood': 'Sol', 'latitude': 40.41476, 'longitude': -3.70418, 'room_type': 'Entire home/apt', 'price': 65, 'minimum_nights': 5, 'number_of_reviews': 172, 'last_review': datetime.datetime(2022, 9, 26, 0, 0), '

In [10]:
# Crear un índice en el campo 'desc_barrio_local' en las otras colecciones
mycol_economica.create_index([('desc_barrio_local', ASCENDING)])
mycol_locales.create_index([('desc_barrio_local', ASCENDING)])
mycol_licencias.create_index([('desc_barrio_local', ASCENDING)])
mycol_terrazas.create_index([('desc_barrio_local', ASCENDING)])

'desc_barrio_local_1'

4. Para mostrar la eficiencia de tu nuevo modelo de datos, construye y ejecuta 
las siguientes consultas: 
a. El total de alojamientos, locales y terrazas por Distrito y Barrio. 
b. Los barrios con mayor número de alojamientos y terrazas con licencias 
concedidas en los últimos dos años (se quiere analizar la posibilidad de 
ampliar el número de alojamientos). 
c. Los alojamientos con más reseñas por barrio y el número de locales 
con actividad comercial cercanos a dichos alojamientos (establece tú 
el umbral para definir “más reseñas”). 
d. Diferenciar los tipos de alojamientos por barrio, teniendo en cuenta el 
precio, reseñas, el número de habitaciones y los servicios de cada 
alojamiento.

a. El total de alojamientos, locales y terrazas por Distrito y Barrio. 

In [17]:
pipeline_a = [
    {
        '$group': {
            '_id': {'distrito': '$desc_distrito_local', 'barrio': '$desc_barrio_local'},
            'total_alojamientos': {'$sum': 1}
        }
    }
]
resultados_a = coleccion_alojamientos.aggregate(pipeline_a)
for res in resultados_a:
    print(res)

b. Los barrios con mayor número de alojamientos y terrazas con licencias 
concedidas en los últimos dos años (se quiere analizar la posibilidad de 
ampliar el número de alojamientos). 

In [18]:
pipeline_b = [
    {
        '$match': {
            'desc_situacion_terraza': 'Concedida',
            'Fecha_confir_ult_decreto_resol': {'$gte': '2022-01-01'}
        }
    },
    {
        '$group': {
            '_id': '$desc_barrio_local',
            'total_terrazas': {'$sum': 1}
        }
    },
    {
        '$sort': {'total_terrazas': -1}
    }
]
resultados_b = mycol_terrazas.aggregate(pipeline_b)

for res in resultados_b:
    print(res)

c. Los alojamientos con más reseñas por barrio y el número de locales 
con actividad comercial cercanos a dichos alojamientos (establece tú 
el umbral para definir “más reseñas”). 

In [19]:
pipeline_c = [
    {
        '$match': {
            'number_of_reviews': {'$gte': 100}  # Establece aquí tu umbral para "más reseñas"
        }
    },
    {
        '$lookup': {
            'from': 'locales',
            'localField': 'neighbourhood',
            'foreignField': 'desc_barrio_local',
            'as': 'locales_cercanos'
        }
    },
    {
        '$project': {
            'neighbourhood': 1,
            'number_of_reviews': 1,
            'num_locales_cercanos': {'$size': '$locales_cercanos'}
        }
    }
]
resultados_c = coleccion_alojamientos.aggregate(pipeline_c)


for res in resultados_c:
    print(res)

{'_id': ObjectId('65c8fbf5ff639b3d17edf29e'), 'neighbourhood': 'Hispanoamérica', 'number_of_reviews': 106, 'num_locales_cercanos': 0}
{'_id': ObjectId('65c8fbf5ff639b3d17edf29f'), 'neighbourhood': 'Sol', 'number_of_reviews': 172, 'num_locales_cercanos': 0}
{'_id': ObjectId('65c8fbf5ff639b3d17edf2a3'), 'neighbourhood': 'Legazpi', 'number_of_reviews': 196, 'num_locales_cercanos': 0}
{'_id': ObjectId('65c8fbf5ff639b3d17edf2a5'), 'neighbourhood': 'Justicia', 'number_of_reviews': 135, 'num_locales_cercanos': 0}
{'_id': ObjectId('65c8fbf5ff639b3d17edf2a7'), 'neighbourhood': 'Universidad', 'number_of_reviews': 142, 'num_locales_cercanos': 0}
{'_id': ObjectId('65c8fbf5ff639b3d17edf2a8'), 'neighbourhood': 'Cortes', 'number_of_reviews': 192, 'num_locales_cercanos': 0}
{'_id': ObjectId('65c8fbf5ff639b3d17edf2a9'), 'neighbourhood': 'Embajadores', 'number_of_reviews': 181, 'num_locales_cercanos': 0}
{'_id': ObjectId('65c8fbf5ff639b3d17edf2aa'), 'neighbourhood': 'Justicia', 'number_of_reviews': 174,

d. Diferenciar los tipos de alojamientos por barrio, teniendo en cuenta el 
precio, reseñas, el número de habitaciones y los servicios de cada 
alojamiento.

In [20]:
pipeline_d = [
    {
        '$group': {
            '_id': {'barrio': '$neighbourhood', 'tipo': '$room_type'},
            'precio_promedio': {'$avg': '$price'},
            'reseñas_promedio': {'$avg': '$number_of_reviews'},
            'habitaciones_promedio': {'$avg': '$bedrooms'},  # Asegúrate de que tienes un campo 'bedrooms' en tu colección
            'servicios_promedio': {'$avg': '$amenities'}  # Asegúrate de que tienes un campo 'amenities' en tu colección
        }
    }
]
resultados_d = coleccion_alojamientos.aggregate(pipeline_d)

for res in resultados_d:
    print(res)

{'_id': {'barrio': 'Pinar del Rey', 'tipo': 'Entire home/apt'}, 'precio_promedio': 129.43636363636364, 'reseñas_promedio': 41.71875, 'habitaciones_promedio': None, 'servicios_promedio': None}
{'_id': {'barrio': 'Castillejos', 'tipo': 'Entire home/apt'}, 'precio_promedio': 345.58490566037733, 'reseñas_promedio': 22.942196531791907, 'habitaciones_promedio': None, 'servicios_promedio': None}
{'_id': {'barrio': 'El Pardo', 'tipo': 'Entire home/apt'}, 'precio_promedio': 237.0, 'reseñas_promedio': 0.0, 'habitaciones_promedio': None, 'servicios_promedio': None}
{'_id': {'barrio': 'Cármenes', 'tipo': 'Shared room'}, 'precio_promedio': 500.0, 'reseñas_promedio': 1.0, 'habitaciones_promedio': None, 'servicios_promedio': None}
{'_id': {'barrio': 'Santa Eugenia', 'tipo': 'Entire home/apt'}, 'precio_promedio': 84.0, 'reseñas_promedio': 61.0, 'habitaciones_promedio': None, 'servicios_promedio': None}
{'_id': {'barrio': 'Bellas Vistas', 'tipo': 'Shared room'}, 'precio_promedio': 45.0, 'reseñas_promed

RETO 3 – VISUALIZA LOS DATOS 


1. Diseña un modelo de grafo que represente los diferentes locales, terrazas y 
alojamientos por cada barrio de Madrid. 
a. Describe los nodos que utilizarías, así como sus atributos. 
b. Propón posibles etiquetas que ayuden a consultar el grafo. 
c. Propón las relaciones que utilizarías, así como sus atributos.

a. Describe los nodos que utilizarías, así como sus atributos.

Nodo Alojamiento: Este nodo representaría cada alojamiento individual en el CSV de ‘Listing’. Los atributos podrían ser "id, name, neighbourhood, room_type"
Nodo Terraza: Este nodo representaría cada terraza individual en el CSV de ‘Terrazas’. Los atributos podrían ser "id_terraza, id_local, desc_distrito_local, rotulo"
Nodo Local: Este nodo representaría cada local individual en el CSV de ‘Locales’. Los atributos podrían ser "id_local, desc_distrito_local, rotulo"

b. Propón posibles etiquetas que ayuden a consultar el grafo. 

Para el csv listing pondria la etiqueta **alojamiento**: Esta etiqueta se podría aplicar a los nodos que representan alojamientos individuales, se podría usarla para realizar consultas que se centren en los alojamientos.
Para el csv locales pondria la etiqueta **local**: Esta etiqueta se podría aplicar a los nodos que representan locales individuales, se podría usarla para realizar consultas que se centren en los locales.
Para el csv terrazas pondria la etiqueta **terraza**: Esta etiqueta se podría aplicar a los nodos que representan terrazas individuales, se podría usar para realizar consultas que se centren en las terrazas.

c. Propón las relaciones que utilizarías, así como sus atributos.

ALOJADO_EN: Esta relación podría conectar los nodos Alojamiento y Local. Si un alojamiento específico está ubicado en un local, se podría crear una relación ALOJADO_EN desde el nodo Alojamiento hasta el nodo Local
TIENE_TERRAZA: Esta relación podría conectar los nodos Local y Terraza. Si un local tiene una terraza, se podría crear una relación TIENE_TERRAZA desde el nodo Local hasta el nodo Terraza
EN_BARRIO: Esta relación podría conectar los nodos Alojamiento, Local y Terraza con nodos Barrio. Si un alojamiento, local o terraza está ubicado en un barrio específico, se podría crear una relación EN_BARRIO desde el nodo Alojamiento, Local o Terraza hasta el nodo Barrio


Voy a utilizar los nodos de Listing, Terrazas y locales

Me he fijado que el CSV esta mal estructurado y hay que hacer un tratamiento de datos para poder leerlo correctamente

Revisando los datos de los CSV uno se puede fijar en que los datos estan mal estructurados para ello hay que hacer un tratammiendo de los datos:

- Eliminando los espacios en blanco
- Eliminar columnas innecesarias
- Cambiar el formato de la fecha
- Cambiar el formato del precio del alquiler

Leer los csv

In [73]:
locales = pd.read_csv("C:/Users/flore/.Neo4jDesktop/relate-data/dbmss/dbms-f5aeb400-b05a-41bf-a760-c08cdb7d6007/import/locales202312.csv", delimiter=";")
terrazas = pd.read_csv("C:/Users/flore/.Neo4jDesktop/relate-data/dbmss/dbms-f5aeb400-b05a-41bf-a760-c08cdb7d6007/import/terrazas202312.csv", delimiter=";")
alojamientos = pd.read_csv("C:/Users/flore/.Neo4jDesktop/relate-data/dbmss/dbms-f5aeb400-b05a-41bf-a760-c08cdb7d6007/import/listings.csv", delimiter=",")


Tratamiento del CSV para poder explotar los datos posteriormente

In [74]:
# Borrar los espacios en blanco de algunas columnas
df_local = locales.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df_local.to_csv('C:/Users/flore/.Neo4jDesktop/relate-data/dbmss/dbms-f5aeb400-b05a-41bf-a760-c08cdb7d6007/import/locales.csv')

df_terraza = terrazas.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df_terraza.to_csv('C:/Users/flore/.Neo4jDesktop/relate-data/dbmss/dbms-f5aeb400-b05a-41bf-a760-c08cdb7d6007/import/terrazas.csv')

columnas = ['id', 'name','neighbourhood','room_type','price','number_of_reviews']
alojamientos = alojamientos[columnas]

# crear una columna que sea numeros de habitaciones
alojamientos['n_bedroom'] = alojamientos['name'].str.extract('(\d+) bedroom')
# Los hoteles no dicen que tengan 0 habitaciones, directamente ni lo muestran, entonces como averiguamos que el hotel no tiene bedroom? porque al extraer el numero de habitacion solo con
# los numeros, aquellos que esten a null serán los que no tengan habitaciones entonces hacemos un fillna a 0
alojamientos['n_bedroom'] = alojamientos['n_bedroom'].fillna('0')



C:\Users\flore\AppData\Local\Temp\ipykernel_20548\1251377290.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_local = locales.applymap(lambda x: x.strip() if isinstance(x, str) else x)
C:\Users\flore\AppData\Local\Temp\ipykernel_20548\1251377290.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_terraza = terrazas.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [85]:
# comprobaciones de que si hay hoteles que no tienen dormitorios
alojamientos[alojamientos['n_bedroom'] == '0']
# llegado a este punto, tampoco necesitamos el nombre de los hoteles, porque en el enunciado no nos pide nombres
alojamientos.drop('name', axis=1)

,id,neighbourhood,room_type,price,number_of_reviews,n_bedroom
0,6369,Hispanoamérica,Private room,85,106,1
1,30320,Sol,Entire home/apt,65,172,1
2,21853,Cármenes,Private room,31,33,1
3,24805,Universidad,Entire home/apt,94,33,6
4,30959,Embajadores,Entire home/apt,0,8,1
...,...,...,...,...,...,...
25538,1046219783290634155,Zofío,Entire home/apt,120,0,2
25539,1045519419976898566,Guindalera,Entire home/apt,120,0,1
25540,1046222300986271054,Castillejos,Entire home/apt,171,0,1
25541,1046239141714406672,Arapiles,Entire home/apt,74,0,2


In [ ]:
alojamientos.to_csv('C:/Users/flore/.Neo4jDesktop/relate-data/dbmss/dbms-f5aeb400-b05a-41bf-a760-c08cdb7d6007/import/listings.csv')

In [3]:
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "123")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

Importo el csv de listings

In [18]:
records, summary, keys = driver.execute_query(
    "LOAD CSV WITH HEADERS FROM 'file:///listings.csv' AS csv_line CREATE (a:Alojamiento { id: toInteger(csv_line.id), name: csv_line.name, neighbourhood: csv_line.neighbourhood, price: csv_line.price})",
    database_="neo4j",
)

C:\Users\flore\AppData\Local\Temp\ipykernel_16244\297114394.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(


Importo el csv de locales

In [38]:
records, summary, keys = driver.execute_query(
    "LOAD CSV WITH HEADERS FROM 'file:///locales.csv' AS row CREATE (l:Local {id_local: toInteger(row.id_local), neighbourhood: row.desc_distrito_local, rotulo: row.rotulo})",
    database_="neo4j",
)

C:\Users\flore\AppData\Local\Temp\ipykernel_18148\624164719.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(


Importo el csv de terrazas

In [39]:
records, summary, keys = driver.execute_query(
    "LOAD CSV WITH HEADERS FROM 'file:///terrazas.csv' AS row CREATE (t:Terraza {id_terraza: toInteger(row.id_terraza), id_local: toInteger(row.id_local),  neighbourhood: row.desc_distrito_local, rotulo: row.rotulo})",
    database_="neo4j",
)

C:\Users\flore\AppData\Local\Temp\ipykernel_18148\964933968.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(


In [8]:
for local in locales:
    print(local)

# Summary information
print("The query `{query}` returned {records_count} records in {time} ms.".format(
    query=summary.query, records_count=len(records),
    time=summary.result_available_after,
))

The query `LOAD CSV WITH HEADERS FROM 'file:///locales202312.csv' AS csv_line FIELDTERMINATOR ';' CREATE (l:locales { id: toInteger(csv_line.id_local), barrio_local: csv_line.desc_barrio_local, cord_x: csv_line.coordenada_x_local, cod_y: csv_line.coordenada_y_local, tipo_acceso_local: csv_line.desc_tipo_acceso_local, situacion_local: csv_line.desc_situacion_local})` returned 0 records in 1583 ms.


Aqui hago las relaciones de los nodos

1 Local puede tener 1 Terraza

In [4]:
records, summary, keys = driver.execute_query(
    "MATCH (l:Local), (t:Terraza) WHERE l.id_local = t.id_local CREATE (l)-[:TIENE]->(t)",
    database_="neo4j",
)

C:\Users\flore\AppData\Local\Temp\ipykernel_16244\2301886817.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(


2. Define algunas consultas sobre tu grafo: 
a. Consulta todos los locales y terrazas del barrio “Salamanca”. 
b. Consulta los alojamientos cuyo precio supere los 100€. 
c. Consulta los barrios donde los alojamientos no cuentan con 
dormitorios. 
d. Consulta los barrios donde los alojamientos no cuentan con reseñas.

a. Consulta todos los locales y terrazas del barrio “Salamanca”. 

In [6]:
# Locales en el barrio Salamanca
records, summary, keys = driver.execute_query(
    "MATCH (l:Local {neighbourhood: 'SALAMANCA'}) RETURN l",
    database_="neo4j",
)

for res in records:
    print(res)

C:\Users\flore\AppData\Local\Temp\ipykernel_16244\1268830467.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(


<Record l=<Node element_id='5' labels=frozenset({'Local'}) properties={'neighbourhood': 'SALAMANCA', 'rotulo': 'AREA MOBILE', 'id_local': 270270399}>>
<Record l=<Node element_id='14' labels=frozenset({'Local'}) properties={'neighbourhood': 'SALAMANCA', 'rotulo': 'MERCADO DE LA PAZ', 'id_local': 270270935}>>
<Record l=<Node element_id='15' labels=frozenset({'Local'}) properties={'neighbourhood': 'SALAMANCA', 'rotulo': 'LA BOITE DEL PINTOR', 'id_local': 270270959}>>
<Record l=<Node element_id='25' labels=frozenset({'Local'}) properties={'neighbourhood': 'SALAMANCA', 'rotulo': 'VENTANAS SAN MIGUEL', 'id_local': 270546353}>>
<Record l=<Node element_id='29' labels=frozenset({'Local'}) properties={'neighbourhood': 'SALAMANCA', 'rotulo': 'SPEJOS PELUQUEROS', 'id_local': 270271171}>>
<Record l=<Node element_id='34' labels=frozenset({'Local'}) properties={'neighbourhood': 'SALAMANCA', 'rotulo': 'RESTAURANTE THONG', 'id_local': 270271336}>>
<Record l=<Node element_id='35' labels=frozenset({'Loca

In [7]:
# Terrazas en el barrio Salamanca
records, summary, keys = driver.execute_query(
    "MATCH (t:Terraza {neighbourhood: 'SALAMANCA'}) RETURN t",
    database_="neo4j",
)

for res in records:
    print(res)

<Record t=<Node element_id='26978' labels=frozenset({'Terraza'}) properties={'id_terraza': 33, 'neighbourhood': 'SALAMANCA', 'rotulo': 'VIPS', 'id_local': 270403150}>>
<Record t=<Node element_id='26980' labels=frozenset({'Terraza'}) properties={'id_terraza': 42, 'neighbourhood': 'SALAMANCA', 'rotulo': 'VIPS', 'id_local': 40002970}>>
<Record t=<Node element_id='26981' labels=frozenset({'Terraza'}) properties={'id_terraza': 1176, 'neighbourhood': 'SALAMANCA', 'rotulo': 'SALITRE', 'id_local': 40002488}>>
<Record t=<Node element_id='26984' labels=frozenset({'Terraza'}) properties={'id_terraza': 57, 'neighbourhood': 'SALAMANCA', 'rotulo': 'CAFETERIA CESAR LAS VENTAS', 'id_local': 40001046}>>
<Record t=<Node element_id='26986' labels=frozenset({'Terraza'}) properties={'id_terraza': 58, 'neighbourhood': 'SALAMANCA', 'rotulo': 'CAFETERIA PARISIENA', 'id_local': 270075832}>>
<Record t=<Node element_id='26992' labels=frozenset({'Terraza'}) properties={'id_terraza': 64, 'neighbourhood': 'SALAMANC

C:\Users\flore\AppData\Local\Temp\ipykernel_16244\81874939.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(


b. Consulta los alojamientos cuyo precio supere los 100€.

In [ ]:
records, summary, keys = driver.execute_query(
    "MATCH (a:Alojamiento) WHERE a.price > '100' RETURN a",
    database_="neo4j",
)

c. Consulta los barrios donde los alojamientos no cuentan con 
dormitorios. 